# kor2vec train.py

## settings

In [1]:
import numpy as np
import tensorflow as tf
import collections
from konlpy.tag import Twitter
import re
import math
import random
import pandas as pd

DATA_PATH = "./data/"

## Pre-process Data test using Our data(poke wiki crawl)

In [2]:
# crawling한 데이터를 불러온다.
pk_data = pd.read_csv(DATA_PATH + 'pk_data_g1.csv')

# 단어 별로 자른다.
desc_list = []
words = []
for desc in pk_data['desc'][0].split('.'):
    desc_list.append(desc)
    sentence = re.sub(r"[^ㄱ-힣a-zA-Z0-9]+", ' ', desc).strip().split()
    if sentence:
        words.append(sentence)
        print(sentence)

['태어날', '때부터', '등에', '식물의', '씨앗이', '있어', '조금씩', '크게', '자란다']
['태어났을', '때부터', '등에', '이상한', '씨앗이', '심어져', '있으며', '몸과', '함께', '자란다고', '한다']
['며칠', '동안', '아무것도', '먹지', '않아도', '건강']
['등의', '씨앗에', '많은', '영양이', '있기에', '괜찮다']
['등의', '씨앗', '안에는', '영양이', '듬뿍']
['씨앗은', '몸과', '같이', '커지고', '있다']
['태어날', '때부터', '등에', '씨앗을', '짊어지고', '있다']
['몸이', '커지면', '자란만큼', '씨앗도', '커지게', '된다']
['태어나서', '잠깐', '동안', '등의', '씨앗에', '모인', '영양을', '받고', '자란다']
['양지에서', '낮잠', '자는', '모습을', '볼', '수', '있다']
['태양의', '빛을', '많이', '받으면', '등의', '씨앗이', '크게', '자란다']
['태어날', '때부터', '등에', '식물의', '씨앗이', '있어', '조금씩', '크게', '자란다']
['태어났을', '때부터', '등에', '이상한', '씨앗이', '심어져', '있으며', '몸과', '함께', '자란다고', '한다']
['태어나면서', '잠깐', '동안', '등의', '씨앗에서', '영양을', '받으며', '크게', '자란다']
['태어나서부터', '얼마', '동안은', '등의', '씨앗으로부터', '영양을', '공급받아', '크게', '성장한다']
['등의', '씨앗', '안에는', '영양이', '가득하다']
['씨앗은', '몸과', '함께', '커진다']
['태어날', '때부터', '등에', '씨앗을', '짊어지고', '있다']
['몸이', '크게', '성장함에', '따라', '씨앗도', '커진다']
['태어나서부터', '얼마', '동안은', '등의', '씨앗으로부터', '영양을', '공급받아', '크게', 

In [3]:
# 단어의 빈도수를 세고 빈도수가 일정 이상인 단어만 저장한다.
word_counter = [['UNK', -1]] # 빈도수 문제로 word_dict에 없는 word를 처리하기 위함. unknown
word_counter.extend(collections.Counter([word for sentence in words for word in sentence]).most_common())
print(word_counter)
word_counter = [item for item in word_counter if item[1] >= 0 or item[0] == 'UNK']
print(word_counter)

# 저장한 단어의 index를 지정하고 저장.
word_dict = dict()
for word, count in word_counter:
    word_dict[word] = len(word_dict)
word_reverse_dict = dict(zip(word_dict.values(), word_dict.keys()))
print(len(word_dict))
print(word_dict)
print(word_reverse_dict)

[['UNK', -1], ('등의', 10), ('크게', 9), ('때부터', 7), ('등에', 7), ('씨앗이', 7), ('자란다', 6), ('몸과', 5), ('있다', 5), ('영양을', 5), ('태어날', 4), ('함께', 4), ('태어났을', 3), ('이상한', 3), ('심어져', 3), ('있으며', 3), ('자란다고', 3), ('한다', 3), ('동안', 3), ('영양이', 3), ('태어나서부터', 3), ('얼마', 3), ('동안은', 3), ('씨앗으로부터', 3), ('공급받아', 3), ('성장한다', 3), ('식물의', 2), ('있어', 2), ('조금씩', 2), ('씨앗에', 2), ('씨앗', 2), ('안에는', 2), ('씨앗은', 2), ('씨앗을', 2), ('짊어지고', 2), ('몸이', 2), ('씨앗도', 2), ('잠깐', 2), ('양지에서', 2), ('낮잠', 2), ('자는', 2), ('모습을', 2), ('볼', 2), ('수', 2), ('태양의', 2), ('빛을', 2), ('많이', 2), ('받으면', 2), ('커진다', 2), ('며칠', 1), ('아무것도', 1), ('먹지', 1), ('않아도', 1), ('건강', 1), ('많은', 1), ('있기에', 1), ('괜찮다', 1), ('듬뿍', 1), ('같이', 1), ('커지고', 1), ('커지면', 1), ('자란만큼', 1), ('커지게', 1), ('된다', 1), ('태어나서', 1), ('모인', 1), ('받고', 1), ('태어나면서', 1), ('씨앗에서', 1), ('받으며', 1), ('가득하다', 1), ('성장함에', 1), ('따라', 1)]
[['UNK', -1], ('등의', 10), ('크게', 9), ('때부터', 7), ('등에', 7), ('씨앗이', 7), ('자란다', 6), ('몸과', 5), ('있다', 5), ('영양을', 5), ('태어날', 4), ('

In [4]:
# 단어들을 형태소 분석기로 분리한다.
word_to_pos_li = dict()
pos_list = list()
twitter = Twitter()
for w in word_dict:
    w_pos_li = list()
    for pos in twitter.pos(w, norm=True):
        w_pos_li.append(pos)
   
    # 단어별로 형태소 분리하고 원래의 index에 맞게 저장.
    word_to_pos_li[word_dict[w]] = w_pos_li
    pos_list += w_pos_li

# print(word_to_pos_li)
# print("pos_list: ", pos_list)

# 분리된 단어들 counting. (굳이 카운팅하는게 필요한지?)
pos_counter = collections.Counter(pos_list).most_common()
# print(pos_counter)

# 형태소 분석기에 의해 분리된 단어들을 각각 index를 지정하고 저장.
pos_dict = dict()
for pos, _ in pos_counter:
    pos_dict[pos] = len(pos_dict)

pos_reverse_dict = dict(zip(pos_dict.values(), pos_dict.keys()))

print(len(pos_dict))
print("pos_dict : ", pos_dict)
print("pos_reverse_dict : ", pos_reverse_dict)

83
pos_dict :  {('씨앗', 'Noun'): 0, ('다', 'Eomi'): 1, ('을', 'Josa'): 2, ('의', 'Josa'): 3, ('이', 'Josa'): 4, ('자란', 'Verb'): 5, ('커지', 'Verb'): 6, ('등', 'Noun'): 7, ('에', 'Josa'): 8, ('몸', 'Noun'): 9, ('영양', 'Noun'): 10, ('며', 'Eomi'): 11, ('동안', 'Noun'): 12, ('태어나서', 'Verb'): 13, ('은', 'Josa'): 14, ('도', 'Josa'): 15, ('에서', 'Josa'): 16, ('받으', 'Verb'): 17, ('면', 'Eomi'): 18, ('에', 'Eomi'): 19, ('UNK', 'Alpha'): 20, ('크게', 'Noun'): 21, ('때', 'Noun'): 22, ('부터', 'Josa'): 23, ('과', 'Josa'): 24, ('있다', 'Adjective'): 25, ('태어날', 'Verb'): 26, ('함께', 'Adverb'): 27, ('태어났', 'Verb'): 28, ('을', 'Eomi'): 29, ('이상한', 'Adjective'): 30, ('심어', 'Verb'): 31, ('져', 'Eomi'): 32, ('있으', 'Adjective'): 33, ('다고', 'Eomi'): 34, ('한', 'Verb'): 35, ('부터', 'Eomi'): 36, ('얼마', 'Noun'): 37, ('으로부터', 'Josa'): 38, ('공급', 'Noun'): 39, ('받아', 'Verb'): 40, ('성장한', 'Verb'): 41, ('식물', 'Noun'): 42, ('있어', 'Adjective'): 43, ('조금씩', 'Adverb'): 44, ('안', 'Noun'): 45, ('에는', 'Josa'): 46, ('짊어지', 'Noun'): 47, ('고', 'Josa'): 4

In [5]:
# 분리된 단어들을 word vector의 index로 매핑하여 분리되기전 index로 저장.
word_to_pos_dict = dict()
print(word_to_pos_li)

for word_id, pos_li in word_to_pos_li.items():
    print(word_id, ' ', pos_li)
    pos_id_li = list()
    for pos in pos_li:
        pos_id_li.append(pos_dict[pos])
    word_to_pos_dict[word_id] = pos_id_li
print(word_to_pos_dict)

{0: [('UNK', 'Alpha')], 1: [('등', 'Noun'), ('의', 'Josa')], 2: [('크게', 'Noun')], 3: [('때', 'Noun'), ('부터', 'Josa')], 4: [('등', 'Noun'), ('에', 'Josa')], 5: [('씨앗', 'Noun'), ('이', 'Josa')], 6: [('자란', 'Verb'), ('다', 'Eomi')], 7: [('몸', 'Noun'), ('과', 'Josa')], 8: [('있다', 'Adjective')], 9: [('영양', 'Noun'), ('을', 'Josa')], 10: [('태어날', 'Verb')], 11: [('함께', 'Adverb')], 12: [('태어났', 'Verb'), ('을', 'Eomi')], 13: [('이상한', 'Adjective')], 14: [('심어', 'Verb'), ('져', 'Eomi')], 15: [('있으', 'Adjective'), ('며', 'Eomi')], 16: [('자란', 'Verb'), ('다고', 'Eomi')], 17: [('한', 'Verb'), ('다', 'Eomi')], 18: [('동안', 'Noun')], 19: [('영양', 'Noun'), ('이', 'Josa')], 20: [('태어나서', 'Verb'), ('부터', 'Eomi')], 21: [('얼마', 'Noun')], 22: [('동안', 'Noun'), ('은', 'Josa')], 23: [('씨앗', 'Noun'), ('으로부터', 'Josa')], 24: [('공급', 'Noun'), ('받아', 'Verb')], 25: [('성장한', 'Verb'), ('다', 'Eomi')], 26: [('식물', 'Noun'), ('의', 'Josa')], 27: [('있어', 'Adjective')], 28: [('조금씩', 'Adverb')], 29: [('씨앗', 'Noun'), ('에', 'Josa')], 30: [('씨앗', 'N

In [6]:
# Sub-sampling frequent words according to sampling_rate
# 빈번하게 나오는 단어를 희석한다. 전체 counter가 많아야 의미가 있는것 같음

def sub_sampling(data, word_counter, word_dict, sampling_rate):
    total_words = sum([len(sentence) for sentence in data])
    # print("total_words: {}".format(total_words))
    prob_dict = dict()
    for word, count in word_counter:
        f = count / total_words # 빈도수가 많을수록 f가 1에 가까워짐.
        p = max(0, 1 - math.sqrt(sampling_rate / f)) # sampling_rate가 0.0001이면 f가 클수록 prob이 커진다.
        prob_dict[word_dict[word]] = p
        # print("count : {}, f : {}, p : {}, prob_dict : {}".format(count, f, p, prob_dict))

    new_data = list()
    for sentence in data:
        s = list()
        for word in sentence:
            prob = prob_dict[word]
            if random.random() > prob: # prob이 작을수록 s에 저장되기 쉬움.
                s.append(word)
        new_data.append(s)

    return new_data

In [7]:
#
data = list()
unk_count = 0
for sentence in words:
    s = list()
    for word in sentence:
        print(word)
        if word in word_dict:
            index = word_dict[word]
        else:
            index = word_dict['UNK']
            unk_count += 1
        s.append(index)
        print(s)
    data.append(s)
print("before : ", word_counter)
word_counter[0][1] = max(1, unk_count)
print("after : ", word_counter)
print(data)

# data = sub_sampling(data, word_counter, word_dict, 0.0001)
# print(data)

태어날
[10]
때부터
[10, 3]
등에
[10, 3, 4]
식물의
[10, 3, 4, 26]
씨앗이
[10, 3, 4, 26, 5]
있어
[10, 3, 4, 26, 5, 27]
조금씩
[10, 3, 4, 26, 5, 27, 28]
크게
[10, 3, 4, 26, 5, 27, 28, 2]
자란다
[10, 3, 4, 26, 5, 27, 28, 2, 6]
태어났을
[12]
때부터
[12, 3]
등에
[12, 3, 4]
이상한
[12, 3, 4, 13]
씨앗이
[12, 3, 4, 13, 5]
심어져
[12, 3, 4, 13, 5, 14]
있으며
[12, 3, 4, 13, 5, 14, 15]
몸과
[12, 3, 4, 13, 5, 14, 15, 7]
함께
[12, 3, 4, 13, 5, 14, 15, 7, 11]
자란다고
[12, 3, 4, 13, 5, 14, 15, 7, 11, 16]
한다
[12, 3, 4, 13, 5, 14, 15, 7, 11, 16, 17]
며칠
[49]
동안
[49, 18]
아무것도
[49, 18, 50]
먹지
[49, 18, 50, 51]
않아도
[49, 18, 50, 51, 52]
건강
[49, 18, 50, 51, 52, 53]
등의
[1]
씨앗에
[1, 29]
많은
[1, 29, 54]
영양이
[1, 29, 54, 19]
있기에
[1, 29, 54, 19, 55]
괜찮다
[1, 29, 54, 19, 55, 56]
등의
[1]
씨앗
[1, 30]
안에는
[1, 30, 31]
영양이
[1, 30, 31, 19]
듬뿍
[1, 30, 31, 19, 57]
씨앗은
[32]
몸과
[32, 7]
같이
[32, 7, 58]
커지고
[32, 7, 58, 59]
있다
[32, 7, 58, 59, 8]
태어날
[10]
때부터
[10, 3]
등에
[10, 3, 4]
씨앗을
[10, 3, 4, 33]
짊어지고
[10, 3, 4, 33, 34]
있다
[10, 3, 4, 33, 34, 8]
몸이
[35]
커지면
[35, 60]
자란만큼
[35, 60, 61]
씨

In [10]:
print(word_reverse_dict[10])
print(word_to_pos_dict[10])
print(pos_reverse_dict[26])

태어날
[26]
('태어날', 'Verb')
